**Generating Hard Negatives for MentalFAQ**

In [12]:
!pip3 install elasticsearch

In [13]:
!pip3 install elasticsearch-dsl

In [14]:
from elasticsearch import Elasticsearch

import sys
sys.path.insert(0, '../../../BERT-FAQ/')

from evaluation import get_relevance_label_df
from evaluation import get_relevance_label
from hard_negatives_generator import Hard_Negatives_Generator

# import utility functions
from shared.utils import load_from_json
from shared.utils import dump_to_json
from shared.utils import make_dirs

In [15]:
query_answer_pair_filepath = '../../../BERT-FAQ/data/MentalFAQ/query_answer_pairs.json'
relevance_label_df = get_relevance_label_df(query_answer_pair_filepath)
relevance_label_df

,label,query_type,question,answer,id,query_string,jc_sim,lv_dist
0,1,faq,What is COVID-19?,COVID-19 is a new coronavirus that we have not...,1,NaN,NaN,NaN
1,1,faq,What is a coronavirus?,Coronavirus are a type of virus - there are ma...,2,NaN,NaN,NaN
2,1,faq,Any advice about how to minimize risk during g...,"Currently, there is no evidence of food or foo...",3,NaN,NaN,NaN
3,1,faq,Why do I have to stay at home?,Socially distancing is a practice that aims to...,4,NaN,NaN,NaN
4,1,faq,What is social distancing and how does it help...,Social distancing is one of the most effective...,5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1466,1,user_query,Will warm weather / summer / heat stop outbrea...,"We do not know. Some viruses, like the common ...",1467,will coronavirus stop in summer,0.2500,0.2500
1467,1,user_query,Will warm weather / summer / heat stop outbrea...,"We do not know. Some viruses, like the common ...",1468,will coronavirus stop in the summer,0.2308,0.2667
1468,1,user_query,Will warm weather / summer / heat stop outbrea...,"We do not know. Some viruses, like the common ...",1469,will coronavirus survive in the summer,0.1429,0.2833
1469,1,user_query,"How long does the virus live on surfaces, like...",A recent study shows that the virus can live i...,1470,will coronavirus survive on surfaces,0.0455,0.2115


In [16]:
es = Elasticsearch([{'host':'localhost','port':9200}], http_auth=('username', 'password')) 

hng = Hard_Negatives_Generator(
    es=es, index='mentalfaq', query_by=['question_answer'], top_k=100, query_type='faq')
hard_negatives_faq = hng.get_hard_negatives(relevance_label_df)
print(hard_negatives_faq)
dump_to_json(hard_negatives_faq, '../../../BERT-FAQ/data/MentalFAQ/hard_negatives_faq.json')

hng = Hard_Negatives_Generator(
    es=es, index='mentalfaq', query_by=['question_answer'], top_k=100, query_type='user_query')
hard_negatives_user_query = hng.get_hard_negatives(relevance_label_df)

print(hard_negatives_user_query)
dump_to_json(hard_negatives_user_query, '../../../BERT-FAQ/data/MentalFAQ/hard_negatives_user_query.json')

ValueError: Grouper for 'query_string' not 1-dimensional